# **LP4_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))



### **Generate Mathematical Model**

In [ ]:
problem = """ Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets.

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs)."""

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Parameters: 

- M1 = yield per acre of wheat = 2.5 tons
- M2 = yield per acre of corn = 3 tons
- M3 = yield per acre of sugar beet = 20 tons
- P1 = selling price per ton of wheat = $170
- P2 = selling price per ton of corn = $150
- P3 = selling price per ton of sugar beet = $36 for 6000 T, P4 = $10 for excess production
- C1 = planting cost per acre of wheat = $150
- C2 = planting cost per acre of corn = $230
- C3 = planting cost per acre of sugar beet = $260
- W = wheat needed for cattle feed = 200 tons
- C = corn needed for cattle feed = 240 tons
- B = sugar beets quota = 6000 T
- A = total land = 500 acres
- P1_buy = purchase price for wheat = $170 * 1.40
- P2_buy = purchase price for corn = $150 * 1.40

Decision Variables: 

- X1 = acres of land used for wheat
- X2 = acres of land used for corn
- X3 = acres of land used for sugar beet

Objective Function: 

Minimize (total costs, including purchasing crops for cattle feed and planting costs) = C1*X1 + C2*X2 + C3*X3 + MAX(0, W - M1*

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

First, we'll need to install pyomo and select your preferred solver:

```
!pip install pyomo
!apt install glpk-utils # for glpk as solver
!apt install coinor-cbc # for cbc as solver
```

Then, here is the python pyomo code for the problem described:

```python
#Importing necessary modules
from pyomo.environ import *
from pyomo.opt import *

#Creating a Concrete Model
model = ConcreteModel()

#Defining decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.X3 = Var(within=NonNegativeReals)

#Parameters
M1 = 2.5
M2 = 3.0
M3 = 20.0
P1 = 170.0
P2 = 150.0
P3 = 36.0
P4 = 10.0
C1 = 150.0
C2 = 230.0
C3 = 260.0
W = 200.0
C = 240.0
B = 6000.0
A = 500.0
P1_buy = P1 * 1.40
P2_buy = P2 * 1.40

#Defining Objective Function
model.Cost = Objective(expr = C1*model.X1 + C2*model.X2 + C3*model.X3 + 
                       (0 <= (W - M1*model.X1))*(W - M1*model.X1)*P1_buy + 
                       (0 <= (C - M2*model.X2))*(C - M2*model.X2)*P2_buy, sense=minimize)

#Constraint 1
model.Constraint1 = Constraint(expr = model.X1 + model.X2 + model.X3 <= A)

#Constraint 2
model.Constraint2 = Constraint(expr = M1*model.X1 >= W)

#Constraint 3
model.Constraint3 = Constraint(expr = M2*model.X2 >= C)

#Constraint 4
model.Constraint4 = Constraint(expr = M3*model.X3 <= B + (0 <= (M3*model.X3 - B))*(M3*model.X3 - B)/2 )

#Running the solver and printing solution
opt = SolverFactory('glpk')
opt.solve(model)

print('Decision Variables:')
print('X1 =', model.X1())
print('X2 =', model.X2())
print('X3 =', model.X3())
```

Sample data are used as defined parameters above. Please note that given configuration for sugar beats quota limit might not be accurate for pyomo constraints, please adjust it according to your correct problem constraints.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *
from pyomo.opt import *

# Creating a Concrete Model
model = ConcreteModel()

# Defining decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.X3 = Var(within=NonNegativeReals)

# Parameters
M1 = 2.5
M2 = 3.0
M3 = 20.0
P1 = 170.0
P2 = 150.0
P3 = 36.0
P4 = 10.0
C1 = 150.0
C2 = 230.0
C3 = 260.0
W = 200.0
C = 240.0
B = 6000.0
A = 500.0
P1_buy = P1 * 1.40
P2_buy = P2 * 1.40

# Defining Objective Function
model.Cost = Objective(expr = C1*model.X1 + C2*model.X2 + C3*model.X3 +
                       (0 <= (W - M1*model.X1))*(W - M1*model.X1)*P1_buy +
                       (0 <= (C - M2*model.X2))*(C - M2*model.X2)*P2_buy, sense=minimize)

# Constraint 1
model.Constraint1 = Constraint(expr = model.X1 + model.X2 + model.X3 <= A)

# Constraint 2
model.Constraint2 = Constraint(expr = M1*model.X1 >= W)

# Constraint 3
model.Constraint3 = Constraint(expr = M2*model.X2 >= C)

# Constraint 4
model.Constraint4 = Constraint(expr = M3*model.X3 <= B + (0 <= (M3*model.X3 - B))*(M3*model.X3 - B)/2 )

# Running the solver and printing solution
opt = SolverFactory('glpk')
opt.solve(model)

print('Decision Variables:')
print('X1 =', model.X1())
print('X2 =', model.X2())
print('X3 =', model.X3())

TypeError: unsupported operand type(s) for *: 'InequalityExpression' and 'LinearExpression'

### **Edit and Run the code for the mathematical model produced by GPT4 (if translation of model into code was incorrect)**

In [ ]:
from pyomo.environ import *
from pyomo.opt import *

# Creating a Concrete Model
model = ConcreteModel()

# Defining decision variables
model.X1 = Var(within=NonNegativeReals)
model.X2 = Var(within=NonNegativeReals)
model.X3 = Var(within=NonNegativeReals)

# Parameters
M1 = 2.5
M2 = 3.0
M3 = 20.0
P1 = 170.0
P2 = 150.0
P3 = 36.0
P4 = 10.0
C1 = 150.0
C2 = 230.0
C3 = 260.0
W = 200.0
C = 240.0
B = 6000.0
A = 500.0
P1_buy = P1 * 1.40
P2_buy = P2 * 1.40
M=1000000

model.b1 = Var(domain=Binary)
model.b2 = Var(domain=Binary)
model.b3 = Var(domain=Binary)

model.objconstraint1 = Constraint(expr= W - (M1*model.X1) <= model.b1 * M)
model.objconstraint2 = Constraint(expr=W - (M1*model.X1) <= M * (1 - model.b1))

model.objconstraint3 = Constraint(expr= C - (M2*model.X2) <= model.b2 * M)
model.objconstraint4 = Constraint(expr=C - (M2*model.X2) <= M * (1 - model.b2))

# Defining Objective Function
model.Cost = Objective(expr = C1*model.X1 + C2*model.X2 + C3*model.X3 + (W - (M1*model.X1))*P1_buy +
                       (C - (M2*model.X2))*P2_buy, sense=minimize)

# Constraint 1
model.Constraint1 = Constraint(expr = model.X1 + model.X2 + model.X3 <= A)

# Constraint 2
model.Constraint2 = Constraint(expr = M1*model.X1 >= W)

# Constraint 3
model.Constraint3 = Constraint(expr = M2*model.X2 >= C)

# Constraint 4

model.consconstraint1 = Constraint(expr= M3*model.X3 - B <= model.b3 * M)
model.consconstraint2 = Constraint(expr= M3*model.X3 - B <= M * (1 - model.b3))

model.Constraint4 = Constraint(expr = M3*model.X3 <= B + (1/2)*(M3*model.X3 - B) )

# Running the solver and printing solution
opt = SolverFactory('glpk')
opt.solve(model)

print('Decision Variables:')
print('X1 =', model.X1())
print('X2 =', model.X2())
print('X3 =', model.X3())
print(model.Cost())

Decision Variables:
X1 = 420.0
X2 = 80.0
X3 = 0.0
-120899.99999999997
